# Union-Find Week-plan

This notebook consolidates all key theory, pseudocode, and Python implementations for the Union-Find (Disjoint Set Union) data structure, covering all topics from the week-plan.

## 1. Quick Find

**Idea**: Maintain an array `id[]` where `id[i]` is the set identifier for element `i`.  
**FIND** is O(1); **UNION** is O(n).

**Pseudocode**
```text
INIT(n):
  for i in 0…n-1:
    id[i] ← i

FIND(i):
  return id[i]

UNION(p, q):
  pid ← FIND(p)
  qid ← FIND(q)
  if pid == qid: return
  for i in 0…n-1:
    if id[i] == pid:
      id[i] ← qid
```

In [ ]:
class QuickFindUF:
    def __init__(self, n):
        self.id = list(range(n))
    def find(self, i):
        return self.id[i]
    def union(self, p, q):
        pid, qid = self.find(p), self.find(q)
        if pid == qid:
            return
        for i in range(len(self.id)):
            if self.id[i] == pid:
                self.id[i] = qid

## 2. Quick Union

**Idea**: Treat `id[]` as parent links; each element points to its parent.  
**FIND** follows parent links to the root (tree); **UNION** makes one root point to the other.  
Worst-case tree height O(n).

**Pseudocode**
```text
INIT(n):
  for i in 0…n-1:
    id[i] ← i

FIND(i):
  while i != id[i]:
    i ← id[i]
  return i

UNION(p, q):
  rootP ← FIND(p)
  rootQ ← FIND(q)
  if rootP == rootQ: return
  id[rootP] ← rootQ
```

In [ ]:
class QuickUnionUF:
    def __init__(self, n):
        self.id = list(range(n))
    def find(self, i):
        while i != self.id[i]:
            i = self.id[i]
        return i
    def union(self, p, q):
        rootP, rootQ = self.find(p), self.find(q)
        if rootP != rootQ:
            self.id[rootP] = rootQ

## 3. Weighted Quick Union

**Idea**: Keep track of tree sizes and always attach the smaller tree under the larger tree to keep height O(log n).

**Pseudocode**
```text
INIT(n):
  for i in 0…n-1:
    id[i] ← i, sz[i] ← 1

FIND(i):
  while i != id[i]:
    i ← id[i]
  return i

UNION(p, q):
  rootP ← FIND(p), rootQ ← FIND(q)
  if rootP == rootQ: return
  if sz[rootP] < sz[rootQ]:
    id[rootP] ← rootQ
    sz[rootQ] += sz[rootP]
  else:
    id[rootQ] ← rootP
    sz[rootP] += sz[rootQ]
```

In [ ]:
class WeightedQuickUnionUF:
    def __init__(self, n):
        self.id = list(range(n))
        self.sz = [1]*n
    def find(self, i):
        while i != self.id[i]:
            i = self.id[i]
        return i
    def union(self, p, q):
        rootP, rootQ = self.find(p), self.find(q)
        if rootP == rootQ:
            return
        if self.sz[rootP] < self.sz[rootQ]:
            self.id[rootP] = rootQ
            self.sz[rootQ] += self.sz[rootP]
        else:
            self.id[rootQ] = rootP
            self.sz[rootP] += self.sz[rootQ]

## 4. Path Compression

**Idea**: During `FIND`, make every examined node point directly to the root to flatten the tree, giving near-constant amortized time.

**Pseudocode (two-pass)**
```text
FIND(i):
  root ← i
  while root != id[root]:
    root ← id[root]
  # second pass
  while i != root:
    next ← id[i]
    id[i] ← root
    i ← next
  return root
```

In [ ]:
def find_with_path_compression(id, i):
    root = i
    while root != id[root]:
        root = id[root]
    # path compression
    while i != root:
        next_i = id[i]
        id[i] = root
        i = next_i
    return root

**Pseudocode (recursive)**
```text
FIND(i):
  if i != id[i]:
    id[i] = FIND(id[i])
  return id[i]
```

In [ ]:
def find_recursive_pc(id, i):
    if i != id[i]:
        id[i] = find_recursive_pc(id, id[i])
    return id[i]

## 5. Alternative Quick Find Variant

**Variant**: In `UNION`, only update `id[i] == id[p]` entries rather than whole array?  
**Comment**: Still O(n) in worst case, functions correctly but no asymptotic improvement over Quick Find.

**Pseudocode**
```text
UNION(p, q):
  pid, qid = FIND(p), FIND(q)
  if pid != qid:
    for k in 0…n-1:
      if id[k] == pid:
        id[k] = qid
```

## 6. Dynamic Connected Components via Graph Search

Use **DFS** or **BFS** on the graph to find connected components in O(n + m) per query or update.  
**Trade-off**: Union-Find supports incremental connectivity in almost-constant amortized time, so much faster for many union/find operations.

**Pseudocode (BFS)**
```text
CONNECTED-COMPONENTS(Adj):
  mark all vertices unvisited
  for u in V:
    if not visited[u]:
      BFS(u)  # labels component
```

In [ ]:
from collections import deque

def connected_components(adj):
    n = len(adj)
    cc = [-1]*n
    label = 0
    for i in range(n):
        if cc[i] == -1:
            queue = deque([i])
            cc[i] = label
            while queue:
                u = queue.popleft()
                for v in adj[u]:
                    if cc[v] == -1:
                        cc[v] = label
                        queue.append(v)
            label += 1
    return cc

## 7. Dynamic Connectivity Operations (ADDCABLE / CONNECTED)

**Use**: Union-Find directly for online connectivity queries.  
**Pseudocode**
```text
ADDCABLE(a, b): UNION(a, b)
CONNECTED(a, b): return FIND(a) == FIND(b)
```

In [ ]:
class UnionFind:
    def __init__(self, n):
        self.uf = WeightedQuickUnionUF(n)
    def add_cable(self, a, b):
        self.uf.union(a, b)
    def connected(self, a, b):
        return self.uf.find(a) == self.uf.find(b)

## 8. Percolation / Zombie Invasion

Model a k×k grid: each cell is a site, connect open sites via Union-Find to detect if top and bottom are connected.  
**Pseudocode**
```text
INIT: create UF on k*k + 2 nodes (virtual top, bottom)
for each open cell (i,j):
  for each neighbor (ni,nj) open:
    UNION(cell(i,j), cell(ni,nj))
  if i == 0: UNION(cell, TOP)
  if i == k-1: UNION(cell, BOTTOM)
PERCOLATES if FIND(TOP) == FIND(BOTTOM)
```

In [ ]:
def percolates(grid):
    k = len(grid)
    uf = UnionFind(k*k + 2)
    TOP, BOTTOM = k*k, k*k+1
    def index(i, j): return i*k + j
    for i in range(k):
        for j in range(k):
            if not grid[i][j]: continue
            idx = index(i, j)
            if i == 0: uf.add_cable(idx, TOP)
            if i == k-1: uf.add_cable(idx, BOTTOM)
            for di, dj in [(1,0),(-1,0),(0,1),(0,-1)]:
                ni, nj = i+di, j+dj
                if 0 <= ni < k and 0 <= nj < k and grid[ni][nj]:
                    uf.add_cable(idx, index(ni,nj))
    return uf.connected(TOP, BOTTOM)

## 9. Union-Find with Linked Lists & Weights

**Idea**: Represent each set by a linked list with head/tail pointers and update by appending smaller list to larger.  
- **INIT**: create n lists of one element.  
- **FIND**: O(1) via element-to-head pointer.  
- **UNION**: append smaller list to larger (O(min(size1, size2))).

**Pseudocode**
```text
INIT(n):
  for i in 0…n-1:
    make list [i] with head=i, tail=i, rep[i]=i, size[i]=1

FIND(i):
  return rep[i]

UNION(p, q):
  rp←FIND(p), rq←FIND(q)
  if rp == rq: return
  if size[rp] < size[rq]: swap(rp, rq)
  # append list rq after rp
  tail[rp].next = head[rq]
  tail[rp] = tail[rq]
  for each x in list rq:
    rep[x] = rp
  size[rp] += size[rq]
```

In [ ]:
class ListNode:
    def __init__(self, val):
        self.val = val
        self.next = None

class UFLinkedList:
    def __init__(self, n):
        self.head = [ListNode(i) for i in range(n)]
        self.tail = list(self.head)
        self.rep = list(range(n))
        self.size = [1]*n
    def find(self, i):
        return self.rep[i]
    def union(self, p, q):
        rp, rq = self.find(p), self.find(q)
        if rp == rq:
            return
        if self.size[rp] < self.size[rq]:
            rp, rq = rq, rp
        # append rq list to rp
        self.tail[rp].next = self.head[rq]
        self.tail[rp] = self.tail[rq]
        node = self.head[rq]
        while node:
            self.rep[node.val] = rp
            node = node.next
        self.size[rp] += self.size[rq]